In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
comments = pd.read_csv(r'C:\Users\cdalal\Youtube\USComments.csv',error_bad_lines = False)

In [ ]:
comments.head()

In [ ]:
comments.isnull().sum()

In [ ]:
comments.dropna(inplace=True)

## how to perform Sentiment Analysis
    
    In short , Sentiment analysis is all about analyszing sentiments of Users

In [ ]:
!pip install textblob

In [ ]:
from textblob import TextBlob

In [ ]:
TextBlob('Logan Paul it s yo big day ‼️‼️‼️	').sentiment

In [ ]:
polarity = []
for comment in comments['comment_text']:
    try:
        polarity.append(TextBlob(comment).sentiment.polarity)
    except:
        polarity.append(0)
    

In [ ]:
comments['polarity'] = polarity

In [ ]:
comments.head()

### Lets perform EDA for the Positve sentences

In [ ]:
comments_positive = comments[comments['polarity']==1]

In [ ]:
comments_negative = comments[comments['polarity']==-1]

In [ ]:
!pip install wordcloud

In [ ]:
from wordcloud import WordCloud,STOPWORDS

In [ ]:
comments_negative['comment_text']

In [ ]:
total_comments = ' '.join(comments_negative['comment_text'])

In [ ]:
total_comments[0:100]

In [ ]:
wordcloud = WordCloud().generate(total_comments)
plt.figure(figsize=(15,5))
plt.imshow(wordcloud)
plt.axis('off')

### Conclusion-->> Users are emphasizing more on Terrible , worst ,horrible ,boring , disgusting etc..

### Lets perform EDA for the Negative sentences

In [ ]:
total_comments2 = ' '.join(comments_positive['comment_text'])

In [ ]:
wordcloud = WordCloud(stopwords=set(STOPWORDS)).generate(total_comments2)
plt.figure(figsize=(15,5))
plt.imshow(wordcloud)
plt.axis('off')

## Perform Emoji's Analysis

In [ ]:
!pip install emoji

In [ ]:
import emoji

In [ ]:
emoji_list=[]
for comment in comments['comment_text']:
    for char in comment:
        if char in emoji.EMOJI_DATA:
            emoji_list.append(char)

In [ ]:
len(emoji_list)

In [ ]:
emoji_list[0:10]

In [ ]:
from collections import Counter

In [ ]:
Counter(emoji_list).most_common(10)

In [ ]:
Counter(emoji_list).most_common(10)[0][0]

In [ ]:
Counter(emoji_list).most_common(10)[1][0]

In [ ]:
emojis=[Counter(emoji_list).most_common(10)[i][0] for i in range(10)]

In [ ]:
emojis

In [ ]:
freqs=[Counter(emoji_list).most_common(10)[i][1] for i in range(10)]

In [ ]:
freqs

In [ ]:
!pip install plotly

In [ ]:
import plotly.graph_objs as go

In [ ]:
from plotly.offline import iplot

In [ ]:
trace = go.Bar(x=emojis,y=freqs)

In [ ]:
iplot([trace])

#### Collect Entire data of Youtube !

In [ ]:
import os

In [ ]:
path = r'C:\Users\cdalal\Youtube\additional_data'

In [ ]:
files = os.listdir(path)

In [ ]:
files

In [ ]:
for i in range (1,len(files),2):
    print(i)

In [ ]:
files_csv = [files[i] for i in range(0,len(files),2)]

In [ ]:
full_df = pd.DataFrame()

for file in files_csv:
    current_df = pd.read_csv(path+'/'+file,encoding='iso-8859-1',error_bad_lines=False)
    
    current_df['country'] = file.split('.')[0][0:2]
    full_df=pd.concat([full_df,current_df])

In [ ]:
full_df.head(4)

In [ ]:
full_df.shape

### Analysing the most liked category

In [ ]:
full_df['category_id'].unique()

In [ ]:
cat = pd.read_csv(r'C:\Users\cdalal\Youtube/category_file.txt')

In [ ]:
cat.reset_index(inplace=True)

In [ ]:
cat.columns = ['Category_id','Category_name']

In [ ]:
cat.set_index('Category_id',inplace = True)

In [ ]:
cat

In [ ]:
dct=cat.to_dict()

In [ ]:
dct['Category_name']

In [ ]:
full_df['category_name'] = full_df['category_id'].map(dct['Category_name'])

In [ ]:
full_df.columns

In [ ]:
full_df.head(4)

In [ ]:
plt.figure(figsize=(12,8))
sns.boxplot(x='category_name',y='likes',data=full_df)
plt.xticks(rotation='vertical')

### whether Audience is engaged or not

In [ ]:
full_df.columns

In [ ]:
full_df['like_rate'] = (full_df['likes']/full_df['views'])*100
full_df['dislike_rate'] = (full_df['dislikes']/full_df['views'])*100
full_df['comment_count_rate'] = (full_df['comment_count']/full_df['views'])*100

In [ ]:
full_df.head(3)

In [ ]:
sns.regplot(data= full_df,x='views',y='likes')

In [ ]:
sns.heatmap(full_df[['views','likes','dislikes']].corr(),annot=True)

# which channel have largest number of trending videos

In [ ]:
full_df.head(3)

In [ ]:
cdf = full_df.groupby('channel_title')['video_id'].count().sort_values(ascending = False).to_frame().reset_index().rename(columns={'video_id':'total_videos'})

In [ ]:
import plotly.express as px

In [ ]:
px.bar(data_frame = cdf[0:20],x='channel_title',y='total_videos')

# Does Punctuation in titles and tags have any relation with views ,likes, dislikes , comments?

In [ ]:
import string

In [ ]:
string.punctuation

In [ ]:
def punc_count(x):
    return len([c for c in x if c in string.punctuation])

In [ ]:
full_df['title'][0]

In [ ]:
text = 'Eminem - Walk On Water (Audio) ft. BeyoncÃ©'

In [ ]:
punc_count(text)

In [ ]:
sample = full_df[0:10000]

In [ ]:
sample['count_punc'] = sample['title'].apply(punc_count)

In [ ]:
sample.head()

In [ ]:
plt.figure(figsize=(12,8))
sns.boxplot(x='count_punc',y='views',data=sample)

In [ ]:
sample['count_punc'].corr(sample['views'])